In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from config.config import CNN_Config
from dataloader.load_data import LoadData
from model.cnn import ExplainableCNN
from trainer.trainer import Trainer

### Data, Loaders and Transforms


In [2]:
# Define transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load Dataset
data = LoadData(transform=transform)
training_data, testing_data = data.load_data("CIFAR10")

# Data Loaders
train_loader = data.data_loaders(training_data, "train")
test_loader = data.data_loaders(testing_data, "test")

# Example visualization
examples = iter(train_loader)
samples, labels = next(examples)
print(samples.shape, labels.shape)

Files already downloaded and verified
Files already downloaded and verified
torch.Size([20, 3, 32, 32]) torch.Size([20])


### Experiment 1


#### Configuration


In [3]:
# Import configs
experiment_1_configs = CNN_Config['model_args']
experiment_1_configs

{'input_size': 3,
 'num_classes': 10,
 'hidden_layers': [64, 128, 256, 128, 64],
 'activation': torch.nn.modules.activation.ReLU,
 'norm_layer': torch.nn.modules.batchnorm.BatchNorm2d,
 'drop_prob': 0.4,
 'max_pool': False}

In [4]:
model = ExplainableCNN(input_size=experiment_1_configs['input_size'], hidden_layers=experiment_1_configs['hidden_layers'], 
                       num_classes=experiment_1_configs['num_classes'], activation=experiment_1_configs['activation'], 
                       normalization=experiment_1_configs['norm_layer'], max_pool=experiment_1_configs['max_pool'], 
                       drop_prob=experiment_1_configs['drop_prob'])

model

ExplainableCNN(
  (layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.4, inplace=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.4, inplace=False)
    (12): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Dropout(p=0.4, inplace=False)
    (16): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1

#### Model Training


In [5]:
criterion = CNN_Config['criterion']
optimizer = torch.optim.Adam(model.parameters(), lr=CNN_Config['learning_rate'])
batch_size = CNN_Config['batch_size']
epochs = CNN_Config['epochs']
experiment = CNN_Config['name']

model_trainer = Trainer(criterion=criterion, optimizer=optimizer, batch_size=batch_size, 
                        epochs=epochs, train_loader=train_loader, model=model, experiment=experiment)

model_trainer.train_model()

Starting Experiment: CNN_Experiment_1
Epoch: 1 / 20, step 100/2500, loss = 2.3327
Epoch: 1 / 20, step 200/2500, loss = 2.3241
Epoch: 1 / 20, step 300/2500, loss = 2.2460
Epoch: 1 / 20, step 400/2500, loss = 2.1917


KeyboardInterrupt: 